In [ ]:
# !pip install pytorch_lightning==0.7.5
# !pip install transformers==2.9.0 
# !pip install sentencepiece
# !pip install datasets

In [34]:
import torch
import pickle
import numpy as np
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForMaskedLM
import os
import random
import datetime
import pickle

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

data_path = "/media/zihao/New Volume1/UMASS/685_e/github/Zihao_branch/data/Danmu_byt5"
print(data_path)

cuda
/media/zihao/New Volume1/UMASS/685_e/github/Zihao_branch/data/Danmu_byt5


In [13]:
tmp_lst = []

for i in ['danmu', 'comment']:
    with open('{}_token.pkl'.format(i), 'rb') as f:
        tmp = pickle.load(f)
        tmp_lst.append(tmp)
    with open('{}_dist.pkl'.format(i), 'rb') as f:
        tmp = pickle.load(f)
        tmp_lst.append(tmp)
        
danmu_token, danmu_dist, comment_token, comment_dist = tmp_lst

In [14]:
print(len(danmu_token), len(comment_token))

337392 48039


In [15]:
print(danmu_dist[0])
print(len(danmu_dist[2]))

[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
126


In [16]:
danmu_label = []
for i in danmu_dist:
    #print(np.where(i==1))
    max = i.max()
    indexs = np.where(i==max)
    if len(indexs) > 1:
        print(i)
        print(indexs)
        break
    # print(indexs[0])
    # print(len(indexs[0]))
    danmu_label.append(indexs[0][0])
    
print(len(danmu_label))
print(len(danmu_dist))


337392
337392


In [17]:
#not every class have associated data(danmu)
print(set(danmu_label))
print(len(set(danmu_label)))

{1, 2, 3, 4, 5, 6, 10, 11, 13, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 26, 28, 29, 30, 31, 32, 33, 35, 36, 37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56, 57, 59, 60, 61, 62, 63, 65, 66, 67, 68, 69, 71, 72, 73, 74, 75, 77, 78, 79, 80, 81, 83, 84, 85, 86, 87, 88, 90, 91, 92, 93, 94, 96, 97, 98, 106, 107, 109, 110, 111, 112, 114, 115, 116, 117, 124}
93


In [18]:
print(type(danmu_token))
random_index = random.randrange(len(danmu_token))

print(random_index)

<class 'list'>
132683


In [8]:
sort_indexs = range(0, len(danmu_token))
print(sort_indexs)
num = int(len(danmu_dist)*0.8)
print(num)
selected_indexs = random.sample(sort_indexs,num)

range(0, 337392)
269913


In [19]:
train_set = []
test_set = []
train_label = []
test_label = []
for ind, token in enumerate(danmu_token):
    if ind in selected_indexs:
        train_set.append(token)
        train_label.append(danmu_label[ind])
    else:
        test_set.append(token)
        test_label.append(danmu_label[ind])
print(len(train_set),len(test_set))
        

269913 67479


In [28]:
print(len(train_set))
print(len(train_label))
print(len(test_set))
print(len(test_label))

269913
269913
67479
67479


In [29]:
sort_indexs = range(0, len(test_set))
print(sort_indexs)
num = int(len(test_set)*0.5)
print(num)
selected_indexs_val = random.sample(sort_indexs,num)

range(0, 67479)
33739


In [30]:
test_new_set = []
test_new_label = []
val_set = []
val_label = []
for ind, token in enumerate(test_set):
    if ind in selected_indexs_val:
        test_new_set.append(token)
        test_new_label.append(test_label[ind])
    else:
        val_set.append(token)
        val_label.append(test_label[ind])

In [31]:
test_set = test_new_set
test_label = test_new_label

In [ ]:
data_path = data_path+"/pkl/"
if not os.path.exists(data_path):
    os.makedirs(data_path)

In [36]:
with open(data_path+'train/danmu_token_single_label_train.pkl'.format(i), 'wb') as f:
    pickle.dump(train_set,f)
with open(data_path+'train/danmu_dist_single_label_train.pkl'.format(i), 'wb') as f:
    pickle.dump(train_label,f)
    
with open(data_path+'test/danmu_token_single_label_test.pkl'.format(i), 'wb') as f:
    pickle.dump(test_set,f)
with open(data_path+'test/danmu_dist_single_label_test.pkl'.format(i), 'wb') as f:
    pickle.dump(test_label,f)

with open(data_path+'val/danmu_token_single_label_val.pkl'.format(i), 'wb') as f:
    pickle.dump(val_set,f)
with open(data_path+'val/danmu_dist_single_label_val.pkl'.format(i), 'wb') as f:
    pickle.dump(val_set,f)
    


In [38]:
with open(data_path + 'train/danmu_token_single_label_train.pkl', 'rb') as f:
    train_pkl = pickle.load(f)
with open(data_path + 'test/danmu_token_single_label_test.pkl', 'rb') as f:
    test_pkl = pickle.load(f)
with open(data_path + 'val/danmu_token_single_label_val.pkl', 'rb') as f:
    val_pkl = pickle.load(f)
print(train_pkl[0])
print(danmu_token[0])
print(len(train_pkl))
print(len(test_pkl))
print(len(val_pkl))

无能狂怒
1人在观看
269913
33739
33740


In [11]:
#load all data to pkl file without separating training or testing

# with open('danmu_token_single_label.pkl'.format(i), 'wb') as f:
#     # for danmu in danmu_token:
#     #     pickle.dump(danmu, f)
#     pickle.dump(danmu_token,f)
# with open('danmu_dist_single_label.pkl'.format(i), 'wb') as f:
#     pickle.dump(danmu_label, f)

In [26]:
# print(len(danmu_token))
# for ind, label in enumerate(danmu_label):
#     branch = ''
#     if ind in selected_indexs:
#         branch = 'train'
#     else:
#         branch = 'test'
#     path = data_path+"/"+branch+"/"+str(label)
#     if not os.path.exists(path):
#         os.makedirs(path)
#     file_name = str(datetime.datetime.now().strftime("%H_%M_%S_%f")) + '.txt'
#     with open(path+'/'+file_name, 'w+') as f:
#         f.write(danmu_token[ind])
        